In [7]:
import numpy as np

from nuplan.common.actor_state.vehicle_parameters import get_pacifica_parameters
from nuplan.common.actor_state.state_representation import Point2D
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor
from nuplan.planning.utils.multithreading.worker_pool import Task

from tqdm import tqdm

from diffusion_planner.data_process.roadblock_utils import route_roadblock_correction
from diffusion_planner.data_process.agent_process import (
agent_past_process,
sampled_tracked_objects_to_array_list,
sampled_static_objects_to_array_list
)
from diffusion_planner.data_process.map_process import get_neighbor_vector_set_map, map_process
from diffusion_planner.data_process.utils import (convert_to_model_inputs,
get_scenario_map,
get_filter_parameters,
sampled_tracked_objects_to_tensor_list,
sampled_tracked_objects_to_tensor
)
from nuplan.common.actor_state.tracked_objects_types import TrackedObjectType
from diffusion_planner.data_process.utils import convert_absolute_quantities_to_relative,_global_state_se2_array_to_local,_global_velocity_to_local
from nuplan.planning.training.preprocessing.utils.agents_preprocessing import (
    AgentInternalIndex,
    EgoInternalIndex,
    sampled_past_ego_states_to_tensor,
    sampled_past_timestamps_to_tensor,
    compute_yaw_rate_from_state_tensors,
    filter_agents_tensor,
    pack_agents_tensor,
    pad_agent_states
)
import os, torch

from diffusion_planner.model.diffusion_utils.sde import SDE, VPSDE_linear

In [52]:
import numpy as np
import torch

traj = np.random.rand(3, 20, 4)  # NumPy 数组
traj = torch.tensor(traj, dtype=torch.float32)  # 转换为 PyTorch 张量
# print(traj.shape)
t = torch.rand(traj.shape[0])
print(t)
# t = torch.tensor(t, dtype=torch.float32)  # 转换为 PyTorch 标量

sde_1 = VPSDE_linear()
mean, std = sde_1.marginal_prob(traj, t)  # 现在不会报错

print(mean.shape, std)

nosie = torch.randn_like(traj)
print(nosie.shape)
print(std.shape)
noisy_traj = mean + std*nosie
theta_norm = torch.sqrt(noisy_traj[:,:,2]**2 + noisy_traj[:,:,3]**2 + 1e-6)
noisy_traj[:,:,2] /= theta_norm
noisy_traj[:,:,3] /= theta_norm
print(type(noisy_traj))

tensor([0.0870, 0.5941, 0.4159])
torch.Size([3, 20, 4]) tensor([[[0.2838]],

        [[0.9858]],

        [[0.9102]]])
torch.Size([3, 20, 4])
torch.Size([3, 1, 1])
<class 'torch.Tensor'>
